In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import cv2 as cv
from tqdm import tqdm
import os
import json

from lac.utils.plotting import plot_path_3d, plot_3d_points
from lac.localization.mono_vo import MonoVisualOdometry

%load_ext autoreload
%autoreload 2

# Feature detection/matching


In [ ]:
data_path = "/home/shared/data_raw/LAC/segmentation/slam_map1_preset1_teleop"

i = 100
I1_path = os.path.join(data_path, "FrontLeft", f"{i}.png")
I2_path = os.path.join(data_path, "FrontLeft", f"{i + 2}.png")
I1 = cv.imread(I1_path, cv.IMREAD_GRAYSCALE)
I2 = cv.imread(I2_path, cv.IMREAD_GRAYSCALE)

# Plot images side by side
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1, cmap="gray")
ax[1].imshow(I2, cmap="gray")
plt.show()

## LightGlue


In [ ]:
from lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd
from lightglue import match_pair, viz2d

In [ ]:
# SuperPoint+LightGlue
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features="superpoint").eval().cuda()  # load the matcher

# load each image as a torch.Tensor on GPU with shape (3,H,W), normalized in [0,1]
image0 = load_image(I1_path).cuda()
image1 = load_image(I2_path).cuda()

feats0, feats1, matches01 = match_pair(extractor, matcher, image0, image1)
matches = matches01["matches"]  # indices with shape (K,2)
points0 = feats0["keypoints"][matches[..., 0]]  # coordinates in image #0, shape (K,2)
points1 = feats1["keypoints"][matches[..., 1]]  # coordinates in image #1, shape (K,2)

In [ ]:
axes = viz2d.plot_images([image0, image1])
viz2d.plot_matches(points0, points1, color="lime", lw=0.2)
viz2d.add_text(0, f"Stop after {matches01['stop']} layers", fs=20)

kpc0, kpc1 = viz2d.cm_prune(matches01["prune0"]), viz2d.cm_prune(matches01["prune1"])
viz2d.plot_images([image0, image1])
viz2d.plot_keypoints([feats0["keypoints"], feats1["keypoints"]], colors=[kpc0, kpc1], ps=10)

## ORB (left) vs FAST (right)


In [ ]:
fast_detector = cv.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
orb_detector = cv.ORB_create(nfeatures=1000)
orb_kp = orb_detector.detect(I1)
orb_kp, orb_des = orb_detector.compute(I1, orb_kp)

fast_kp = fast_detector.detect(I1)

I1_orb = cv.drawKeypoints(I1, orb_kp, None, color=(0, 255, 0), flags=0)
I1_fast = cv.drawKeypoints(I1, fast_kp, None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_orb)
ax[1].imshow(I1_fast)
plt.show()

In [ ]:
I2_fast = cv.drawKeypoints(I2, fast_detector.detect(I2), None, color=(0, 255, 0), flags=0)
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(I1_fast)
ax[1].imshow(I2_fast)
plt.show()

# Tracking (pyslam-based VO)


In [ ]:
from lac.perception.vision import StereoVIO
from lac.params import FL_X, STEREO_BASELINE, CAMERA_INTRINSICS

In [ ]:
svio = StereoVIO(FL_X, STEREO_BASELINE)

data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/NavAgent/map1_preset4_gtnav_steer")

In [ ]:
# Process the first stereo pair to initialize depths
I1_path = os.path.join(data_path, "FrontLeft", "100.png")
I2_path = os.path.join(data_path, "FrontRight", "100.png")
I1 = cv.imread(I1_path, cv.IMREAD_GRAYSCALE)
I2 = cv.imread(I2_path, cv.IMREAD_GRAYSCALE)
svio.process_stereo_pair(I1, I2)

# Track from frame to frame
for i in tqdm(np.arange(102, 200, 2)):
    I1_path = os.path.join(data_path, "FrontLeft", f"{i}.png")
    I1 = cv.imread(I1_path, cv.IMREAD_GRAYSCALE)

    # Estimate relative pose
    rvec, tvec = svio.track_frame(I1, CAMERA_INTRINSICS)
    print(f"Frame {i}: rvec={rvec.ravel()}, tvec={tvec.ravel()}")

# Mono VO class


In [ ]:
## Parameters ---------------------------------------------------------------

data_path = os.path.expanduser(
    "~/LunarAutonomyChallenge/output/LocalizationAgent/map1_preset0_4m_spiral"
)
img_path = f"{data_path}/FrontLeft"
json_data = json.load(open(f"{data_path}/data_log.json"))
initial_pose = np.array(json_data["initial_pose"])  # TODO: initialize with initial pose

trajlen = 1000

# for KITTI
focal = 915.0
pp = (1280 / 2, 720 / 2)  # principal point

# Parameters for lucas kanade optical flow
lk_params = dict(
    winSize=(21, 21), criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01)
)

In [ ]:
poses = []
positions = []
for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    positions.append(np.array(frame["pose"])[:3, 3])
positions = np.array(positions)

In [ ]:
vo = MonoVisualOdometry(img_path, poses, focal, pp, lk_params)
# vo.R = initial_pose[:3, :3]
# vo.t = initial_pose[:3, 3][:,None]
vo.initialize_pose(initial_pose[:3, :3], initial_pose[:3, 3][:, None])
vo.init_frame(id=84)
vo_traj = np.zeros((trajlen, 3))

for i in tqdm(range(trajlen)):
    vo.process_frame()
    vo_traj[i, :] = vo.get_mono_coordinates()

In [ ]:
fig = plot_path_3d(vo_traj, color="orange", name="VO")
fig = plot_path_3d(positions[:trajlen], color="blue", name="ground truth", fig=fig)
fig.show()